In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [79]:
"""
Bitcoin Mining: Solve complex puzzles to validate transactions and add them to blockchain

puzzles involve computing SHA-256 Hashes over and over

1 Hash = 1 attempt to solve the puzzle

Hashrate = number of hashes/second


Antminer = 110 TH/s which is 110 trillion guesses per second

More hashrate = more revenue but more electricity usage


"""

'\nBitcoin Mining: Solve complex puzzles to validate transactions and add them to blockchain\n\npuzzles involve computing SHA-256 Hashes over and over\n\n1 Hash = 1 attempt to solve the puzzle\n\nHashrate = number of hashes/second\n\n\nAntminer = 110 TH/s which is 110 trillion guesses per second\n\nMore hashrate = more revenue but more electricity usage\n\n\n'

In [81]:
import requests

def get_btc_price():
    # url = "https://api.coingecko.com/api/v3/simple/price"
    # params = {
    #     'ids': 'bitcoin',
    #     'vs_currencies': 'usd'
    # }
    # response = requests.get(url, params=params)
    # print(response.json())
    # data = response.json()
    # return data['bitcoin']['usd']
    btc_price = 105_000
    return btc_price

# Example usage
btc_price = get_btc_price()
print(f"Current BTC Price: ${btc_price}")

Current BTC Price: $105000


In [82]:
def load_clean_NSP_NW_da_data():
    da = pd.read_parquet('data/DA_MISO_NSP_NWELOAD.parquet')
    da['interval_start_local'] = pd.to_datetime(da['interval_start_local'])
    da['interval_end_local'] = pd.to_datetime(da['interval_end_local'])
    da['interval_start_utc'] = pd.to_datetime(da['interval_start_utc'])
    da.sort_values('interval_start_local', inplace=True)
    return da

In [83]:
da = load_clean_NSP_NW_da_data()

In [84]:
def load_clean_NSP_NW_rt_data():
    rt = pd.read_parquet('data/RT_MISO_NSP_NWRT.parquet')
    rt['interval_start_local'] = pd.to_datetime(rt['interval_start_local'])
    rt['interval_end_local'] = pd.to_datetime(rt['interval_end_local'])
    rt['interval_start_utc'] = pd.to_datetime(rt['interval_start_utc'])
    rt.sort_values('interval_start_local', inplace=True)

    return rt


In [85]:
rt = load_clean_NSP_NW_rt_data()
rt.head()

,interval_start_local,interval_start_utc,interval_end_local,interval_end_utc,market,location,location_type,lmp,energy,congestion,loss
0,2021-12-19 23:00:00-06:00,2021-12-20 05:00:00+00:00,2021-12-19 23:05:00-06:00,2021-12-20 05:05:00+00:00,REAL_TIME_5_MIN_FINAL,NSP.NWELOAD,Loadzone,27.30,33.62,-6.86,0.54
1,2021-12-19 23:05:00-06:00,2021-12-20 05:05:00+00:00,2021-12-19 23:10:00-06:00,2021-12-20 05:10:00+00:00,REAL_TIME_5_MIN_FINAL,NSP.NWELOAD,Loadzone,21.55,31.09,-10.17,0.63
2,2021-12-19 23:10:00-06:00,2021-12-20 05:10:00+00:00,2021-12-19 23:15:00-06:00,2021-12-20 05:15:00+00:00,REAL_TIME_5_MIN_FINAL,NSP.NWELOAD,Loadzone,21.34,31.50,-10.58,0.42
3,2021-12-19 23:15:00-06:00,2021-12-20 05:15:00+00:00,2021-12-19 23:20:00-06:00,2021-12-20 05:20:00+00:00,REAL_TIME_5_MIN_FINAL,NSP.NWELOAD,Loadzone,22.30,32.67,-10.88,0.51
4,2021-12-19 23:20:00-06:00,2021-12-20 05:20:00+00:00,2021-12-19 23:25:00-06:00,2021-12-20 05:25:00+00:00,REAL_TIME_5_MIN_FINAL,NSP.NWELOAD,Loadzone,20.59,33.81,-13.73,0.51


In [86]:
def load_btc_data():
    x = pd.read_parquet('data/btc_data.parquet')
    x.rename(columns={'Unnamed: 0':'time_end'}, inplace=True)
    x['time_end'] = pd.to_datetime(x['time_end'])
    x.drop(columns=['time_open','time_close'], inplace=True)
    # x['time_open'] = pd.to_datetime(x['time_open'])
    # x['time_close'] = pd.to_datetime(x['time_close'])
    x = x.iloc[1:]
    return x


In [87]:
df = load_btc_data()

In [88]:
#Assumptions
btc_data = load_btc_data() #history of btc_data
btc_price = get_btc_price() #price btcusd current
power_consumption =3250 #watts
hashrate = 110 #TH/s
num_antminer = 1000
rf = 0.04 #annualized rf rate


rt = load_clean_NSP_NW_da_data()
da = load_clean_NSP_NW_rt_data()

btc_day = 0.00008


breakeven = 95 #$/MWH when btc 93k


In [89]:
def calc_breakeven():
    watts =3250
    watt_to_megawatt = 1000000
    hours = 24
    megawatt_per_hour = watts * hours / watt_to_megawatt
    #print(f"MWH: {megawatt_per_hour}")
    btc_price = get_btc_price()
    btc_gen= 0.00008 #per day,
    dv = btc_price* btc_gen
    #print(f"Dollar Value: {dv}")
    breakeven = dv/ megawatt_per_hour
    return breakeven

In [ ]:
"""
Questions to answer:

- Estimate BTC/USD Volatility

Strategy to be profitable?
- Forecast profits for July 2025 - June 2026
- Forecast some weather
- Location of facility
- Purchase power in DA and if not used sell in RT
- Or purchase less power in DA if need more buy in RT

"""

In [ ]:
"""
Across 500 simulation what is the expected average annual excess profit (profit - rf rate)
- forecast energy prices for 1 year
- forecast btc prices for 1 year
-

What is the volatility in annual profitability
What is the sharpe


Would you buy the facility for 500k?
-Base this off of the return given the volatility of BTC versus electricity
- What is the breakeven expected value
"""

In [73]:
"""
Intraday Analysis
- Choose an hourly intraday period as well as a shorter intraday period
- for reahc period what is the value of operating the BTC facility? What was the value of curtailment? What was the net payout?

-Model BTC and Electricity volatility surface


"""

'\nIntraday Analysis\n- Choose an hourly intraday period as well as a shorter intraday period\n- for reahc period what is the value of operating the BTC facility? What was the value of curtailment? What was the net payout?\n\n-Model BTC and Electricity volatility surface\n\n\n'

In [74]:
#part 2 change the node

In [75]:
# Per minerstat S19s pros consume 3250W of power to produce 110 TH/s and
# generate 0.00008 BTC per day giving them an approximate breakeven of
# $95/MWH at a BTC price of 93k

In [87]:
#input
watts =3250
watt_to_megawatt = 1000000
hours = 24
btc_gen= 0.00008 #per day,
megawatt_per_hour = watts * hours / watt_to_megawatt
print(f"MWH: {megawatt_per_hour}")

MWH: 0.078


In [81]:
btc_price = get_btc_price()
btc_gen= 0.00008 #per day,
dv = btc_price* btc_gen
print(f"Dollar Value: {dv}")

Dollar Value: 8.33744


In [9]:
breakeven = calc_breakeven()
print(f"Breakeven: {breakeven}")

Breakeven: 107.6923076923077


In [102]:
breakeven = dv/ megawatt_per_hour
print(f"Breakeven: {breakeven}")

#if energy is more expensive than the break even we are at a loss/ else
#basically if btc price goes up we can spend more on energy



Breakeven: 106.89025641025643
